In [1]:
#imports
import pandas as pd
import pandas as pd
import ipywidgets as widgets
import math
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
#!pip install bertopic datasets accelerate bitsandbytes xformers adjustText
import pandas as pd

In [2]:
# imports
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.optimization.optimizer import Optimizer
optimizer=Optimizer()
import time
import en_core_web_sm
nlp = en_core_web_sm.load()
dataset = Dataset()

/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
from data import DataLoader

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#Normal Loader
dataloader = DataLoader(dataset="EN").prepare_docs(save="EN.txt").preprocess_octis(output_folder="EN")

100%|██████████| 46236/46236 [00:00<00:00, 115295.43it/s]


created vocab
127126
words filtering done


In [5]:
#DTM Loader
dataloader = DataLoader(dataset="EN_dtm").prepare_docs(save="EN_dtm.txt").preprocess_octis(output_folder="EN_dtm")

100%|██████████| 46236/46236 [00:00<00:00, 114438.17it/s]


created vocab
127126
words filtering done


In [6]:
from sentence_transformers import SentenceTransformer

# Prepare data
dataset, custom = "EN_dtm", True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(data, show_progress_bar=True)

Batches:   0%|          | 0/1445 [00:00<?, ?it/s]

In [7]:
# Match indices
import os
os.listdir(f"./{dataset}")
with open(f"./{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
timestamps = [timestamp for index, timestamp in enumerate(timestamps) if index in indices]
len(data), len(timestamps)

(46236, 46236)

In [8]:
from evaluation import Trainer
params = {
        "nr_topics": 10,
        "min_topic_size": 5,
        "verbose": True,
    }

trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings,
                      custom_dataset=custom,
                      bt_timestamps=timestamps,
                      topk=5,
                      bt_nr_bins=5,
                      verbose=True)
results = trainer.train(f"DynamicBERTopic_EN")

2024-05-14 12:01:20.650051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 12:01:21.113304: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 12:01:35,108 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-14 12:02:20,197 - BERTopic - Dimensionality - Completed ✓
2024-05-14 12:02:20,199 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disab

marjo


In [4]:
from evaluation import Trainer

2024-05-10 17:06:02.465915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 17:06:02.875432: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## LDAseq

In [5]:
import os
import pandas as pd

# Prepare data
dataset, custom = "NL_dtm", True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

# Match indices
os.listdir(f"{dataset}")
with open(f"{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
indices_test = {index: True for index in indices}
timestamps = [timestamp for index, timestamp in enumerate(timestamps) if indices_test.get(index)]
len(data), len(timestamps)

(19058, 19058)

In [6]:
#LDA_seq speedup
#!pip uninstall gensim -y
#!pip install git+https://github.com/RaRe-Technologies/gensim.git@refs/pull/3172/merge


In [ ]:
params = {
    "num_topics": 10,
    "nr_bins": 10,
    "random_state": 42
}

trainer = Trainer(dataset=dataset,
                  model_name="LDAseq",
                  params=params,
                  custom_dataset=custom,
                  bt_timestamps=timestamps,
                  topk=5,
                  verbose=True)
results = trainer.train(f"LDASeq_NL")

19058 19058 [ 521 1060 1104  934 1278  969 1224 1560 3521 6887]


/home/pspaargaren/.local/lib/python3.10/site-packages/gensim/models/ldaseqmodel.py:288: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
